In [4]:
!pip install torch torchvision torchaudio

In [5]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, StandardScaler 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset 
 
# Load dataset 
df = pd.read_csv("/Users/mirkamoltemirkulov/Computational_Intelligence/Crop_recommendation.csv") 
 
# Features and labels 
X = df.drop("label", axis=1) 
y = df["label"] 
 
# Encode labels 
label_encoder = LabelEncoder() 
y_encoded = label_encoder.fit_transform(y) 
 
# Scale features 
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X) 
 
# Train-test split 
X_train, X_test, y_train, y_test = train_test_split( 
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded 
) 
 
# Convert to tensors 
X_train_t = torch.tensor(X_train, dtype=torch.float32) 
X_test_t  = torch.tensor(X_test, dtype=torch.float32) 
y_train_t = torch.tensor(y_train, dtype=torch.long) 
y_test_t  = torch.tensor(y_test, dtype=torch.long) 
 
# Dataloader 
train_dataset = TensorDataset(X_train_t, y_train_t) 
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) 
 
# ANN Model 
class ANNModel(nn.Module): 
    def __init__(self, input_dim, num_classes): 
        super().__init__() 
        self.network = nn.Sequential( 
            nn.Linear(input_dim, 64), 
            nn.ReLU(), 
            nn.Linear(64, 32), 
            nn.ReLU(), 
            nn.Linear(32, num_classes) 
        ) 
 
    def forward(self, x): 
        return self.network(x) 
 
model = ANNModel(X_train.shape[1], len(np.unique(y_encoded))) 
 
# Loss & optimizer 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 
 
# Training loop 
epochs = 30 
for epoch in range(epochs): 
    for features, labels in train_loader: 
        optimizer.zero_grad() 
        outputs = model(features) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
 
# Evaluation 
with torch.no_grad(): 
    outputs = model(X_test_t) 
    _, y_pred = torch.max(outputs, 1) 
 
from sklearn.metrics import classification_report, confusion_matrix 
print(classification_report(y_test, y_pred)) 
print(confusion_matrix(y_test, y_pred)) 
 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        20
           6       0.91      1.00      0.95        20
           7       1.00      1.00      1.00        20
           8       0.94      0.85      0.89        20
           9       1.00      1.00      1.00        20
          10       1.00      1.00      1.00        20
          11       1.00      0.90      0.95        20
          12       1.00      1.00      1.00        20
          13       1.00      1.00      1.00        20
          14       1.00      1.00      1.00        20
          15       1.00      1.00      1.00        20
          16       1.00      1.00      1.00        20
          17       1.00    